In [488]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [489]:
# today's date string for file naming automation
# today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
today_date_str = '_2024-03-07.csv'

print(today_date_str)

_2024-03-07.csv


In [490]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1645 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,MSFT,Microsoft Corporation,NASDAQ,3029388743942.5767,USD,407.7000,USD,401.40,USD,4.14,USD,1.029620,-0.171603,7318938817.36,6038168,72897450,153732,18317632.3,22653584.93333334,24695687.58888889,1.521480,1.802997,1.456598,406.050000,36.87356987,Technology Services,1.33390343,0.9529795,1.0291758,0.8906199,0.006283,0.000648,0.102547,0.069518,0.511495,1.35116077,0.72525565,0.29577401,0.69349908,0.17078787,420.820,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,407.7700,USD,407.7840,USD,407.7840,USD,408.2100,USD,408.2100,USD,408.2100,USD,408.2100,USD,417.35,USD,417.35,USD,958891.40271493,USD,407.44680,407.458012,407.66144,406.709715,408.077949,407.557865,407.037781,408.834448,405.770310,402.706172
1,NVDA,NVIDIA Corporation,NASDAQ,2294911000000,USD,917.9644,USD,889.17,USD,14.55,USD,1.640361,0.244645,2399492500,24983933,196861538,963981,50731960.4,52039927.09999999,45684760.52222235,3.669561,3.465403,3.660522,911.288167,76.90398358,Electronic Technology,1.55029011,1.8023118,2.3419523,1.7245569,0.003040,0.000376,0.169068,0.108818,0.714603,1.8226155,0.93532542,0.43920408,0.87974948,0.24321959,919.880,USD,919.88,USD,919.88,USD,919.88,USD,919.88000,USD,917.7846,USD,918.6000,USD,918.6000,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.88,USD,919.88,USD,2058175.67567568,USD,917.04892,917.041488,917.50708,914.952654,919.480428,917.509580,915.538732,922.645829,911.518965,900.392101
2,AMZN,"Amazon.com, Inc.",NASDAQ,1826433893442.8735,USD,175.8320,USD,172.50,USD,1.32,USD,0.760763,-0.582099,9227833763.2,10147300,116848089,216491,36604066.00000006,47647839.56666667,48387894.11111113,1.818962,1.727769,1.783302,175.407300,60.84994463,Retail Trade,1.18742275,1.1343786,1.3817025,1.1703976,0.002072,0.000255,0.231699,0.153234,0.740425,1.57087472,0.68843918,0.24672143,0.54233525,0.13228522,180.140,USD,180.14,USD,180.14,USD,180.14,USD,188.65391,USD,175.8100,USD,175.9699,USD,175.9699,USD,176.6699,USD,176.6699,USD,176.6699,USD,176.6699,USD,180.14,USD,180.14,USD,376908.19672131,USD,175.75702,175.806900,176.03200,175.929615,176.618810,176.080100,175.541390,177.031126,175.433745,173.836364
3,GOOGL,Alphabet Inc.,NASDAQ,1667862074561.7778,USD,133.6100

In [491]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [492]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,MSFT,Microsoft Corporation,NASDAQ,3.029389e+12,USD,407.7000,USD,401.40,USD,4.14,USD,1.029620,-0.171603,7318938817.36,6038168,72897450,153732,18317632.3,22653584.93333334,24695687.58888889,1.521480,1.802997,1.456598,406.050000,36.87356987,Technology Services,1.33390343,0.9529795,1.0291758,0.8906199,0.006283,0.000648,0.102547,0.069518,0.511495,1.35116077,0.72525565,0.29577401,0.69349908,0.17078787,420.820,USD,420.82,USD,420.82,USD,420.82,USD,420.82000,USD,407.7700,USD,407.7840,USD,407.7840,USD,408.2100,USD,408.2100,USD,408.2100,USD,408.2100,USD,417.35,USD,417.35,USD,958891.40271493,USD,407.44680,407.458012,407.66144,406.709715,408.077949,407.557865,407.037781,408.834448,405.770310,402.706172,Titans
1,NVDA,NVIDIA Corporation,NASDAQ,2.294911e+12,USD,917.9644,USD,889.17,USD,14.55,USD,1.640361,0.244645,2399492500,24983933,196861538,963981,50731960.4,52039927.09999999,45684760.52222235,3.669561,3.465403,3.660522,911.288167,76.90398358,Electronic Technology,1.55029011,1.8023118,2.3419523,1.7245569,0.003040,0.000376,0.169068,0.108818,0.714603,1.8226155,0.93532542,0.43920408,0.87974948,0.24321959,919.880,USD,919.88,USD,919.88,USD,919.88,USD,919.88000,USD,917.7846,USD,918.6000,USD,918.6000,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.8800,USD,919.88,USD,919.88,USD,2058175.67567568,USD,917.04892,917.041488,917.50708,914.952654,919.480428,917.509580,915.538732,922.645829,911.518965,900.392101,Titans
2,AMZN,"Amazon.com, Inc.",NASDAQ,1.826434e+12,USD,175.8320,USD,172.50,USD,1.32,USD,0.760763,-0.582099,9227833763.2,10147300,116848089,216491,36604066.00000006,47647839.56666667,48387894.11111113,1.818962,1.727769,1.783302,175.407300,60.84994463,Retail Trade,1.18742275,1.1343786,1.3817025,1.1703976,0.002072,0.000255,0.231699,0.153234,0.740425,1.57087472,0.68843918,0.24672143,0.54233525,0.13228522,180.140,USD,180.14,USD,180.14,USD,180.14,USD,188.65391,USD,175.8100,USD,175.9699,USD,175.9699,USD,176.6699,USD,176.6699,USD,176.6699,USD,176.6699,USD,180.14,USD,180.14,USD,376908.19672131,USD,175.75702,175.806900,176.03200,175.929615,176.618810,176.080100,175.541390,177.031126,175.433745,173.836364,Titans
3,GOOGL,Alphabet Inc.,NASDAQ,1.667862e

In [493]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [494]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1645 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,377,22.917933
1,Large caps,370,22.492401
2,Midlers,316,19.209726
3,Shrimp,277,16.838906
4,Micro caps,250,15.197568
5,Titans,40,2.431611
6,Undefined,15,0.911854


In [495]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

1630 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
407,CTLT,"Catalent, Inc.",NYSE,1.029120e+10,USD,56.9400,USD,57.24,USD,0.59,USD,1.041483,1.041483,175048909.974,439282,9564708,400,2563576.3,3777591.83333333,2733550.25555556,0.669958,0.832096,1.399961,56.896667,-,Health Technology,1.08684703,1.6171272,1.1340333,1.2011527,0.021211,0.006982,0.644037,0.380304,0.474715,0.91080825,0.38315134,0.16607729,0.70225546,0.18698305,59.280,USD,60.2000,USD,60.200,USD,71.335,USD,142.64,USD,56.9400,USD,56.9850,USD,56.9850,USD,57.0300,USD,57.03,USD,57.03,USD,57.03,USD,57.44,USD,57.73,USD,239494.38202247,USD,56.95000,56.963125,56.97000,56.966462,57.022135,56.98075,56.939365,57.032372,56.945205,56.858038,Large caps
1357,LAES,SEALSQ Corp,NASDAQ,4.827681e+07,USD,1.9525,USD,2.00,USD,0.04,USD,2.051282,2.564103,10534247.286024,186683,3357591,25096,1446861.3,3321972.26666667,4631167.41111111,3.626943,10.047381,17.951749,1.960833,5.53429705,Electronic Technology,0.38405624,3.912784,4.152772,4.152772,0.253404,0.691786,0.103953,0.058988,0.155448,0.75878946,0.2743293,0.11822048,0.08313321,0.11732337,3.950,USD,3.9500,USD,5.000,USD,28.500,USD,28.50,USD,1.9599,USD,1.9599,USD,1.9599,USD,1.9800,USD,2.00,USD,2.00,USD,2.00,USD,2.05,USD,2.05,USD,-,-,1.95260,1.953025,1.95650,1.960808,1.980134,1.96098,1.941826,1.983638,1.960250,1.936862,Shrimp
16,ASML,ASML Holding N.V. - New York Registry Shares,NASDAQ,3.922822e+11,USD,1034.0000,USD,998.40,USD,12.17,USD,1.212236,-0.550835,396778243.329072,527854,4524818,13307,987823.6,1197698.76666667,1065757.72222222,3.131590,2.951517,2.215092,1028.176667,48.08497182,Electronic Technology,1.91794031,1.5979712,1.9130507,1.4806403,0.068890,0.016943,0.202449,0.149947,0.597834,1.71642091,0.78314771,0.47421008,0.7951914,0.28168664,1035.710,USD,1035.7100,USD,1035.710,USD,1035.710,USD,1035.71,USD,1034.8500,USD,1035.7100,USD,1035.7100,USD,1035.7100,USD,1035.71,USD,1035.71,USD,1035.71,USD,1035.71,USD,1035.71,USD,702421.83008627,USD,1035.01138,1034.964612,1034.64536,1032.182062,1035.512638,1034.66234,1033.812042,1038.696882,1029.109305,1019.521728,Titans
452,LYFT,"Lyft, Inc.",NASDAQ,7.276480e+09,USD,18.2000,USD,18.38,USD,0.15,USD,0.819672,0.437158,338664386.32,5032177,69870130,190150,1

In [496]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0    Small caps    377   23.128834
1    Large caps    370   22.699387
2       Midlers    316   19.386503
3        Shrimp    277   16.993865
4    Micro caps    250   15.337423
5        Titans     40    2.453988


In [497]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [498]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 1630
Number of rows removed: 1626
NEW DataFrame length: 4


In [499]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Large caps,1,25.0
1,Midlers,1,25.0
2,Small caps,1,25.0
3,Shrimp,1,25.0


In [500]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Large caps' 'Midlers' 'Small caps' 'Shrimp']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'R

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,AGR,"Avangrid, Inc.",NYSE,1.398403e+10,USD,36.1550,USD,33.60,USD,3.72,USD,11.596010,4.738155,6.853858e+07,2556937,4898242,162256,890896.7,799781.7,923876.26666667,11.818182,2.209133,2.215244,36.070000,17.80859029,Utilities,44.310116,0.89043146,0.7404226,0.54155004,0.008215,0.001919,0.466285,0.451788,1.195366,19.27370505,11.25233605,3.305672,1.44226988,0.31088383,36.3050,USD,36.3050,USD,36.3050,USD,41.2200,USD,57.240,USD,36.1500,USD,36.1750,USD,36.1750,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,1040005.00062508,USD,36.15900,36.161250,36.17500,36.169231,36.240054,36.173250,36.106446,36.284194,36.121300,35.958406,Large caps
1,ARHS,"Arhaus, Inc.",NASDAQ,2.100035e+09,USD,14.9700,USD,13.41,USD,1.18,USD,9.204368,4.602184,3.849788e+07,1844451,5509621,18588,886492.2,729934.2,905800.26666667,20.072464,6.089707,3.844914,14.786667,14.85413773,Consumer Durables,21.535353,2.1525056,2.3993912,2.3993912,0.034688,0.008684,0.187786,0.159855,0.742167,6.67563224,4.8590718,2.410472,1.59420442,0.34527383,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.590,USD,14.9800,USD,15.1100,USD,15.1100,USD,15.1400,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,579683.01886792,USD,15.02900,15.020625,14.95300,14.964377,15.121820,14.952750,14.783680,15.263195,14.929125,14.595055,Midlers
2,ML,MoneyLion Inc.,NYSE,7.195275e+08,USD,69.9800,USD,53.84,USD,9.45,USD,17.982873,2.454805,4.834745e+06,336986,791941,16874,128262.4,113884.2,133646.32222222,31.351256,9.312434,6.593099,66.417467,-,Technology Services,12.848809,2.4364288,3.0196884,2.6366432,0.047272,0.013522,0.312359,0.280489,1.454853,10.71768588,6.43124759,3.300665,1.26885093,0.45998353,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,387.387,USD,70.1300,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,503316.10044313,USD,70.15236,70.327675,69.68292,68.237277,71.266676,69.390585,67.514494,72.638218,66.512145,60.386072,Small caps
3,LYT,Lytus Technologies Holdings PTV. Ltd.,NASDAQ,3.606651e+06,USD,13.5074,USD,6.96,USD,7.14,USD,309.090909,201.298701,3.210263e+05,37786097,42444620,5882429,4454003.6,1607600.

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [501]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [502]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,AGR,NYSE,Large caps
1,ARHS,NASDAQ,Midlers
2,ML,NYSE,Small caps
3,LYT,NASDAQ,Shrimp


In [503]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [504]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '5m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 4/4 [00:00<00:00, 1552.01it/s]


In [505]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-10-09
END date: 2024-03-07
104 days of historical stock data.

416 = Length of dataframe
4 = Unique stocks in dataframe
104.0 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['AGR', 'ARHS', 'ML', 'LYT']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,30.15,30.18,29.300,29.44,2023-10-09,AGR,1705993.0,HISTORICAL_PRICES,AGR,,2023-10-09,1701468989000,0.000000,0.000000,29.44,30.15,30.18,29.300,1705993,29.0267,29.7267,29.7563,28.8886,1705993.0,"Oct 9, 23",0.00,0.0000,AGR
1,31.38,31.53,30.370,30.37,2023-10-10,AGR,2092242.0,HISTORICAL_PRICES,AGR,,2023-10-10,1701468982000,0.040796,0.040796,30.37,31.38,31.53,30.370,2092242,29.9436,30.9394,31.0873,29.9436,2092242.0,"Oct 10, 23",1.23,0.0408,AGR
2,31.73,31.81,30.975,31.59,2023-10-11,AGR,1576963.0,HISTORICAL_PRICES,AGR,,2023-10-11,1701468981000,0.052405,0.052405,31.59,31.73,31.81,30.975,1576963,31.1465,31.2845,31.3634,30.5401,1576963.0,"Oct 11, 23",0.35,0.0112,AGR
3,30.63,31.72,30.240,31.62,2023-10-12,AGR,1566835.0,HISTORICAL_PRICES,AGR,,2023-10-12,1701468986000,0.015920,0.015920,31.62,30.63,31.72,30.240,1566835,31.1761,30.2000,31.2747,29.8154,1566835.0,"Oct 12, 23",-1.10,-0.0347,AGR
4,30.98,31.21,30.740,30.90,2023-10-13,AGR,881421.0,HISTORICAL_PRICES,AGR,,2023-10-13,1701468986000,0.027529,0.027529,30.90,30.98,31.21,30.740,881421,30.4662,30.5451,30.7718,30.3084,881421.0,"Oct 13, 23",0.35,0.0114,AGR


In [506]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [507]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
# and explicitly copy the dataframe to ensure it's not a view but a new object
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]].copy()

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df


,symbol,date,close,high,low,open,volume
0,AGR,2024-03-07,36.24,36.3050,35.7500,35.800,5317215.0
1,ARHS,2024-03-07,14.37,15.5900,13.8000,14.000,3614594.0
2,LYT,2024-03-07,8.70,18.6216,8.0500,9.516,75316515.0
3,ML,2024-03-07,67.36,70.8623,58.4101,59.460,764686.0


In [508]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 416


In [509]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
  Symbol Exchange marketCapType
0    AGR     NYSE    Large caps
1   ARHS   NASDAQ       Midlers
2     ML     NYSE    Small caps
3    LYT   NASDAQ        Shrimp

2a - IEX HISTORICAL data (START):
  symbol        date    close     high      low     open     volume
0    AGR  2023-10-09  30.1500  30.1800  29.3000  29.4400  1705993.0
1     ML  2023-10-09  20.9800  21.5000  20.2550  20.6300    50700.0
2    LYT  2023-10-09   0.1489   0.1568   0.1448   0.1562   555785.0
3   ARHS  2023-10-09   9.4200   9.4500   9.0600   9.2500   473175.0
4    AGR  2023-10-10  31.3800  31.5300  30.3700  30.3700  2092242.0

2b - IEX HISTORICAL data (END):
    symbol        date  close     high      low    open      volume
411    AGR  2024-03-06  32.08  32.3900  31.9200  32.100    659052.0
412   ARHS  2024-03-07  14.37  15.5900  13.8000  14.000   3614594.0
413    AGR  2024-03-07  36.24  36.3050  35.7500  35.800   5317215.0
414     ML  2024-03-07  67.36  70.8623  58.4101 

In [510]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
print(f'U.S. stocks returned from the pre-market gap up screener = {len(tv_L2_df.index)}')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):
U.S. stocks returned from the pre-market gap up screener = 4


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,AGR,"Avangrid, Inc.",NYSE,1.398403e+10,USD,36.1550,USD,33.60,USD,3.72,USD,11.596010,4.738155,6.853858e+07,2556937,4898242,162256,890896.7,799781.7,923876.26666667,11.818182,2.209133,2.215244,36.070000,17.80859029,Utilities,44.310116,0.89043146,0.7404226,0.54155004,0.008215,0.001919,0.466285,0.451788,1.195366,19.27370505,11.25233605,3.305672,1.44226988,0.31088383,36.3050,USD,36.3050,USD,36.3050,USD,41.2200,USD,57.240,USD,36.1500,USD,36.1750,USD,36.1750,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,36.3050,USD,1040005.00062508,USD,36.15900,36.161250,36.17500,36.169231,36.240054,36.173250,36.106446,36.284194,36.121300,35.958406,Large caps
1,ARHS,"Arhaus, Inc.",NASDAQ,2.100035e+09,USD,14.9700,USD,13.41,USD,1.18,USD,9.204368,4.602184,3.849788e+07,1844451,5509621,18588,886492.2,729934.2,905800.26666667,20.072464,6.089707,3.844914,14.786667,14.85413773,Consumer Durables,21.535353,2.1525056,2.3993912,2.3993912,0.034688,0.008684,0.187786,0.159855,0.742167,6.67563224,4.8590718,2.410472,1.59420442,0.34527383,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.590,USD,14.9800,USD,15.1100,USD,15.1100,USD,15.1400,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,15.5900,USD,579683.01886792,USD,15.02900,15.020625,14.95300,14.964377,15.121820,14.952750,14.783680,15.263195,14.929125,14.595055,Midlers
2,ML,MoneyLion Inc.,NYSE,7.195275e+08,USD,69.9800,USD,53.84,USD,9.45,USD,17.982873,2.454805,4.834745e+06,336986,791941,16874,128262.4,113884.2,133646.32222222,31.351256,9.312434,6.593099,66.417467,-,Technology Services,12.848809,2.4364288,3.0196884,2.6366432,0.047272,0.013522,0.312359,0.280489,1.454853,10.71768588,6.43124759,3.300665,1.26885093,0.45998353,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,387.387,USD,70.1300,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,70.8623,USD,503316.10044313,USD,70.15236,70.327675,69.68292,68.237277,71.266676,69.390585,67.514494,72.638218,66.512145,60.386072,Small caps
3,LYT,Lytus Technologies Holdings PTV. Ltd.,NASDAQ,3.606651e+06,USD,13.5074,USD,6.96,USD,7.14,USD,309.090909,201.298701,3.210263e+05,37786097,42444620,5882429,4454003.6,1607600.

In [511]:
"""PICK UP HERE WHERE YOU LEFT OFF"""
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.


'PICK UP HERE WHERE YOU LEFT OFF'

| KEY LEVELS (Support & Resistance) |
|-|

in order of priority of in scope. 
thinking of moving some of the levels out as i test them and see what they look like.
for instance i am thinking of eventually completely replacing MAs with EMAs

In [512]:
"calculate key levels from historical data"
# save historical data to a csv
chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

chart_historical_df.head()

,symbol,date,close,high,low,open,volume
0,AGR,2023-10-09,30.1500,30.1800,29.3000,29.4400,1705993.0
1,ML,2023-10-09,20.9800,21.5000,20.2550,20.6300,50700.0
2,LYT,2023-10-09,0.1489,0.1568,0.1448,0.1562,555785.0
3,ARHS,2023-10-09,9.4200,9.4500,9.0600,9.2500,473175.0
4,AGR,2023-10-10,31.3800,31.5300,30.3700,30.3700,2092242.0


In [513]:
"""
IMPORTANT:
for effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). 
"""
chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
chart_historical_df.set_index('date', inplace=True)

# view the multi-indexed dataframe
chart_historical_df.head()

,symbol,close,high,low,open,volume
date,,,,,,
2023-10-09,AGR,30.1500,30.1800,29.3000,29.4400,1705993.0
2023-10-09,ML,20.9800,21.5000,20.2550,20.6300,50700.0
2023-10-09,LYT,0.1489,0.1568,0.1448,0.1562,555785.0
2023-10-09,ARHS,9.4200,9.4500,9.0600,9.2500,473175.0
2023-10-10,AGR,31.3800,31.5300,30.3700,30.3700,2092242.0


In [514]:
"""SIMPLE MOVING AVERAGES (SMA)"""
def calculate_sma(dataframe, period):
    return dataframe['close'].rolling(window=period).mean()

chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15
date,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333


In [515]:
"""EXPONENTIAL MOVING AVERAGES (EMA)"""
def calculate_ema(dataframe, period):
    return dataframe['close'].ewm(span=period, adjust=False).mean()

# calculate EMAs for the specified periods
chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50
date,,,,,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667,24.529071,24.925685,24.930650,24.455153
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333,20.662714,22.744652,23.787731,24.003186
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333,31.291809,28.163806,26.526995,25.122669
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333,21.631206,23.463114,24.220614,24.228054
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667,18.694137,21.528002,23.134842,23.780680
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667,23.156092,23.446547,23.986761,24.106143
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333,20.227394,21.796266,23.070879,23.724334
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000,25.564930,24.422399,24.325081,24.215144
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333,39.496620,32.229236,28.423645,25.907099


In [516]:
"""MOVING AVERAGE CONVERGENCE DIVERGENCE (MACD)"""
# calculate the MACD line and the Signal line
chart_historical_df['MACD'] = calculate_ema(chart_historical_df, 12) - calculate_ema(chart_historical_df, 26)
chart_historical_df['MACD_Signal'] = chart_historical_df['MACD'].ewm(span=9, adjust=False).mean()

# MACD Histogram -> useful for visualizing momentum and potential buy/sell signals
chart_historical_df['MACD_Histogram'] = chart_historical_df['MACD'] - chart_historical_df['MACD_Signal']

chart_historical_df.tail(10)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram
date,,,,,,,,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667,24.529071,24.925685,24.930650,24.455153,0.144558,0.424959,-0.280400
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333,20.662714,22.744652,23.787731,24.003186,-0.827072,0.174553,-1.001624
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333,31.291809,28.163806,26.526995,25.122669,1.581675,0.455977,1.125698
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333,21.631206,23.463114,24.220614,24.228054,-0.556904,0.253401,-0.810305
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667,18.694137,21.528002,23.134842,23.780680,-1.387679,-0.074815,-1.312864
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667,23.156092,23.446547,23.986761,24.106143,-0.486346,-0.157121,-0.329225
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333,20.227394,21.796266,23.070879,23.724334,-1.187395,-0.363176,-0.824219
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000,25.564930,24.422399,24.325081,24.215144,0.021498,-0.286241,0.307739
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333,39.496620,32.229236,28.423645,25.907099,3.450898,0.461187,2.989711


In [517]:
#### DEV - DATA ORGANIZATION ####
chart_historical_df.columns

Index(['symbol', 'close', 'high', 'low', 'open', 'volume', 'SMA5', 'SMA10',
       'SMA15', 'EMA5', 'EMA10', 'EMA20', 'EMA50', 'MACD', 'MACD_Signal',
       'MACD_Histogram'],
      dtype='object')

| J. Welles Wilder Jr (technical indicators) |
|-|
| (4); ATR, RSI. ADI, Parabolic SAR |
| START |

In [518]:
"""RELATIVE STRENGTH INDEX (RSI)"""
def calculate_rsi(dataframe, periods=14):
    close_delta = dataframe['close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    ma_up = up.rolling(window=periods).mean()
    ma_down = down.rolling(window=periods).mean()

    rsi = ma_up / (ma_up + ma_down) * 100
    return rsi

# Adding the RSI to your DataFrame
chart_historical_df['RSI'] = calculate_rsi(chart_historical_df)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,RSI
date,,,,,,,,,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667,24.529071,24.925685,24.930650,24.455153,0.144558,0.424959,-0.280400,52.106710
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333,20.662714,22.744652,23.787731,24.003186,-0.827072,0.174553,-1.001624,51.144825
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333,31.291809,28.163806,26.526995,25.122669,1.581675,0.455977,1.125698,50.284723
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333,21.631206,23.463114,24.220614,24.228054,-0.556904,0.253401,-0.810305,46.947745
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667,18.694137,21.528002,23.134842,23.780680,-1.387679,-0.074815,-1.312864,51.069095
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667,23.156092,23.446547,23.986761,24.106143,-0.486346,-0.157121,-0.329225,47.673434
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333,20.227394,21.796266,23.070879,23.724334,-1.187395,-0.363176,-0.824219,50.157163
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000,25.564930,24.422399,24.325081,24.215144,0.021498,-0.286241,0.307739,50.626410
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333,39.496620,32.229236,28.423645,25.907099,3.450898,0.461187,2.989711,52.059181


In [519]:
"""Average Directional Index (ADX)"""

'Average Directional Index (ADX)'

In [520]:
"""AVERAGE TRUE RANGE (ATR)"""
def calculate_atr(dataframe, period=14):
    high_low = dataframe['high'] - dataframe['low']
    high_close = (dataframe['high'] - dataframe['close'].shift()).abs()
    low_close = (dataframe['low'] - dataframe['close'].shift()).abs()

    # Calculate the True Range (TR) as the maximum of the three measures
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate the ATR by taking the rolling mean of the TR
    atr = tr.rolling(window=period).mean()
    
    return atr

# Adding the ATR to your DataFrame
chart_historical_df['ATR'] = calculate_atr(chart_historical_df)

chart_historical_df.tail(10)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,RSI,ATR
date,,,,,,,,,,,,,,,,,,
2024-03-05,AGR,31.77,32.4050,31.6300,32.140,675359.0,23.829,27.8025,25.211667,24.529071,24.925685,24.930650,24.455153,0.144558,0.424959,-0.280400,52.106710,32.048536
2024-03-05,ARHS,12.93,13.1400,12.5100,12.690,1245225.0,20.007,23.9095,25.192333,20.662714,22.744652,23.787731,24.003186,-0.827072,0.174553,-1.001624,51.144825,32.665679
2024-03-06,ML,52.55,52.6600,50.3500,51.110,192143.0,29.992,27.8545,28.518333,31.291809,28.163806,26.526995,25.122669,1.581675,0.455977,1.125698,50.284723,32.023407
2024-03-06,LYT,2.31,2.5000,2.2600,2.370,4434724.0,20.398,24.9725,25.336333,21.631206,23.463114,24.220614,24.228054,-0.556904,0.253401,-0.810305,46.947745,34.248407
2024-03-06,ARHS,12.82,13.2500,12.6050,13.110,1631813.0,22.476,21.2485,24.115667,18.694137,21.528002,23.134842,23.780680,-1.387679,-0.074815,-1.312864,51.069095,32.989200
2024-03-06,AGR,32.08,32.3900,31.9200,32.100,659052.0,22.538,23.1835,26.047667,23.156092,23.446547,23.986761,24.106143,-0.486346,-0.157121,-0.329225,47.673434,30.904200
2024-03-07,ARHS,14.37,15.5900,13.8000,14.000,3614594.0,22.826,21.4165,23.548333,20.227394,21.796266,23.070879,23.724334,-1.187395,-0.363176,-0.824219,50.157163,29.435986
2024-03-07,AGR,36.24,36.3050,35.7500,35.800,5317215.0,19.564,24.7780,25.091000,25.564930,24.422399,24.325081,24.215144,0.021498,-0.286241,0.307739,50.626410,29.713486
2024-03-07,ML,67.36,70.8623,58.4101,59.460,764686.0,32.574,26.4860,27.506333,39.496620,32.229236,28.423645,25.907099,3.450898,0.461187,2.989711,52.059181,30.636950


In [521]:
chart_historical_df

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,RSI,ATR
date,,,,,,,,,,,,,,,,,,
2023-10-09,AGR,30.1500,30.1800,29.3000,29.4400,1.705993e+06,NaN,NaN,NaN,30.150000,30.150000,30.150000,30.150000,0.000000,0.000000,0.000000,NaN,NaN
2023-10-09,ML,20.9800,21.5000,20.2550,20.6300,5.070000e+04,NaN,NaN,NaN,27.093333,28.482727,29.276667,29.790392,-0.731510,-0.146302,-0.585208,NaN,NaN
2023-10-09,LYT,0.1489,0.1568,0.1448,0.1562,5.557850e+05,NaN,NaN,NaN,18.111856,23.331122,26.502594,28.627981,-2.958034,-0.708648,-2.249386,NaN,NaN
2023-10-09,ARHS,9.4200,9.4500,9.0600,9.2500,4.731750e+05,NaN,NaN,NaN,15.214570,20.801827,24.875680,27.874727,-3.929177,-1.352754,-2.576423,NaN,NaN
2023-10-10,AGR,31.3800,31.5300,30.3700,30.3700,2.092242e+06,18.41578,NaN,NaN,20.603047,22.725131,25.495139,28.012188,-2.893473,-1.660898,-1.232575,NaN,NaN
2023-10-10,ML,22.2900,22.7600,21.0300,21.0300,8.919400e+04,16.84378,NaN,NaN,21.165365,22.646017,25.189888,27.787789,-2.774177,-1.883554,-0.890623,NaN,NaN
2023-10-10,ARHS,9.5900,9.7900,9.3350,9.4300,5.216770e+05,14.56578,NaN,NaN,17.306910,20.272195,23.704184,27.074150,-3.662203,-2.239284,-1.422919,NaN,NaN
2023-10-10,LYT,0.1733,0.1880,0.1550,0.1600,1.528635e+06,14.57066,NaN,NaN,11.595706,16.617851,21.463148,26.019215,-5.067405,-2.804908,-2.262497,NaN,NaN
2023-10-11,ML,22.3600,22.6900,21.8500,22.1700,3.803600e+04,17.15866,NaN,NaN,15.183804,17.661878,21.548562,25.875716,-4.340719,-3.112070,-1.228649,NaN,NaN


In [522]:
"""PARABOLIC SAR (PSAR)"""

'PARABOLIC SAR (PSAR)'

| J. Welles Wilder Jr (technical indicators) |
|-|
| END |

In [523]:
"""BOLLINGER BANDS"""

'BOLLINGER BANDS'

In [524]:
# LINE LEVELS not full calculation dataframe
# part 1 was calculations append
# part 2 is data points append
# part 3 extra - could be plotting past signals (might integrate ai in here like tensorflow)

| CHARTS |
|-|

# DELL FOCUS - Pre-market

In [525]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [526]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [527]:
# dell_df.head(5)

In [528]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [529]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [530]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [531]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [532]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [533]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [534]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [535]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)